# API Integration & Prompt Automation
Automating Few-Shot and Chain-of-Thought Workflows

## 1. Environment Setup

In [ ]:
!pip install openai anthropic python-dotenv pydantic

In [ ]:
import os
import json
from dotenv import load_dotenv
from pydantic import BaseModel, ValidationError
import openai
from anthropic import Anthropic
from google.colab import userdata

# Store secrets (do this once)
userdata.set('OPENAI_API_KEY', 'your-openai-key-here')
userdata.set('ANTHROPIC_API_KEY', 'your-anthropic-key-here')

## 2. Configuration

In [ ]:
# Load environment variables
load_dotenv()

# Initialize clients
openai_client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
anthropic_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

## 3. Example Management

In [ ]:
%%writefile examples.json
{
    "reviews": [
        {
            "text": "The food was excellent but service was slow",
            "analysis": {
                "rating": 4,
                "main_points": ["great food", "slow service"]
            }
        }
    ],
    "math": [
        {
            "problem": "If a restaurant has 30 tables with 4 chairs each, how many customers can they seat?",
            "solution": "1. Calculate total chairs: 30 tables * 4 chairs = 120 chairs\n2. Each chair seats 1 customer\n3. Total capacity: 120 customers"
        }
    ]
}

In [ ]:
with open("examples.json") as f:
    EXAMPLES = json.load(f)

## 4. Core Functions

In [ ]:
class ReviewAnalysis(BaseModel):
    rating: int
    main_points: list[str]
    recommendation: bool

In [ ]:
def classify_task(text: str) -> str:
    """Classify input as review or math problem"""
    math_keywords = ['calculate', 'solve', 'how many', 'math', 'problem']
    if any(kw in text.lower() for kw in math_keywords):
        return 'math'
    return 'review'

In [ ]:
def analyze_with_openai(text: str, task_type: str):
    """Handle OpenAI API calls with error handling"""
    try:
        if task_type == 'review':
            examples = "\n".join([f"Review: {ex['text']}\nAnalysis: {ex['analysis']}"
                                 for ex in EXAMPLES['reviews']])
            prompt = f"""Analyze this review:\n{text}\n\nExamples:\n{examples}\n\nOutput JSON:"""

            response = openai_client.chat.completions.create(
                model="model="gpt-4o-mini-2024-07-18",
                messages=[{"role": "user", "content": prompt}],
                response_format={ "type": "json_object" }
            )
            return json.loads(response.choices[0].message.content)

        else:  # Math problem
            examples = "\n".join([f"Problem: {ex['problem']}\nSolution: {ex['solution']}"
                                 for ex in EXAMPLES['math']])
            prompt = f"""Solve this problem step-by-step:\n{text}\n\nExamples:\n{examples}\n\nSolution:"""

            response = openai_client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                temperature=0
            )
            return response.choices[0].message.content

    except Exception as e:
        print(f"OpenAI Error: {str(e)}")
        return None

In [ ]:
def analyze_with_claude(text: str, task_type: str):
    """Handle Anthropic API calls"""
    try:
        if task_type == 'review':
            examples = "\n".join([f"Review: {ex['text']}\nAnalysis: {ex['analysis']}"
                                 for ex in EXAMPLES['reviews']])
            prompt = f"""Analyze this review:\n{text}\n\nExamples:\n{examples}\n\nOutput JSON:"""

            response = anthropic_client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=1000,
                messages=[{"role": "user", "content": prompt}]
            )
            return json.loads(response.content[0].text)

        else:  # Math problem
            examples = "\n".join([f"Problem: {ex['problem']}\nSolution: {ex['solution']}"
                                 for ex in EXAMPLES['math']])
            prompt = f"""Solve this problem step-by-step:\n{text}\n\nExamples:\n{examples}\n\nSolution:"""

            response = anthropic_client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=1000,
                messages=[{"role": "user", "content": prompt}],
                temperature=0
            )
            return response.content[0].text

    except Exception as e:
        print(f"Claude Error: {str(e)}")
        return None

## 5. Main Execution Flow

In [ ]:
def validate_review(response):
    """Validate review analysis output"""
    try:
        return ReviewAnalysis(**response)
    except ValidationError as e:
        print(f"Validation Error: {str(e)}")
        return None

In [ ]:
def main():
    text = input("Enter text to analyze: ")
    task_type = classify_task(text)

    print(f"\nAnalyzing as {task_type} task...\n")

    # Get results from both models
    openai_result = analyze_with_openai(text, task_type)
    claude_result = analyze_with_claude(text, task_type)

    # Validate and display results
    if task_type == 'review':
        print("OpenAI Result:")
        validated = validate_review(openai_result)
        print(validated.dict() if validated else "Invalid response")

        print("\nClaude Result:")
        validated = validate_review(claude_result)
        print(validated.dict() if validated else "Invalid response")
    else:
        print("OpenAI Solution:")
        print(openai_result)
        print("\nClaude Solution:")
        print(claude_result)

In [ ]:
# Run the system
if __name__ == "__main__":
    main()

## 6. Homework Extensions

1. Add Cohere API support
2. Implement caching
3. Add FastAPI wrapper
4. Handle ambiguous inputs

Example cache implementation starter code:

```python
from functools import lru_cache

@lru_cache(maxsize=100)
def cached_analysis(text: str, model: str):
    # Add cache logic here
```